# TODO: 
# REVERT BINARY LABELS ENTIRELY TO LOOK FOR IMPROVEMENT IN TEST DATA ACCURACY

# Add drop out and make the architecture more complex

In [2]:
import numpy as np
import nibabel as nib
import os
import tensorflow as tf
from PIL import Image
import PIL
import skimage.transform as skTrans

In [3]:
aut_path = "DATASET\\AUTISTIC_TRAIN\\"
cnt_path = "DATASET\\CONTROL_TRAIN\\"

In [28]:
test_aut_path = "DATASET\\AUTISTIC_VAL\\"
test_cnt_path = "DATASET\\CONTROL_VAL\\"

In [5]:
def to_arr(path):
    return np.array(nib.load(path).dataobj)

In [6]:
len(os.listdir(cnt_path))

80

In [7]:
labels = []
for i in range(len(os.listdir(aut_path))):
    labels.append(0)
for i in range(len(os.listdir(cnt_path))):
    labels.append(1)
labels = np.array(labels)

In [8]:
labels.shape

(160,)

In [9]:
img = to_arr(aut_path+'2anat.nii.gz')
img.shape

(256, 175, 256)

In [10]:
tf.image.resize(img, [150,200]).numpy().shape

(150, 200, 256)

In [11]:
auts = []
cnts = []

for i in os.listdir(aut_path):
    im = nib.load(aut_path+i).get_fdata()
    #arr = to_arr(aut_path+i)
    r = skTrans.resize(im, (150,200,150), order=1, preserve_range=True)#.reshape(150,200,150,1)
    #print(r.shape)
    #break
    #if arr.shape[2] != 150:
    #    #print(arr.shape)
    #    arr = arr[:,:,46:-60]
    print(i)
    auts.append(r)
for i in os.listdir(cnt_path): 
    #arr = to_arr(cnt_path+i)
    #if arr.shape[2] != 150:
    #    #print(arr.shape)
    #    arr = arr[:,:,46:-60]
    #    #print(arr.shape)
    im = nib.load(cnt_path+i).get_fdata()
    r = skTrans.resize(im, (150,200,150), order=1, preserve_range=True)#.reshape(150,200,150,1)
    print(i)
    cnts.append(r)
    #cnts.append(to_arr(cnt_path+i).reshape(193,256, 256))

10anat.nii.gz
11anat.nii.gz
12anat.nii.gz
13anat.nii.gz
14anat.nii.gz
15anat.nii.gz
16anat.nii.gz
17anat.nii.gz
18anat.nii.gz
19anat.nii.gz
1anat.nii.gz
20anat.nii.gz
21anat.nii.gz
22anat.nii.gz
23anat.nii.gz
24anat.nii.gz
25anat.nii.gz
26anat.nii.gz
27anat.nii.gz
28anat.nii.gz
29anat.nii.gz
2anat.nii.gz
30anat.nii.gz
31anat.nii.gz
32anat.nii.gz
33anat.nii.gz
34anat.nii.gz
35anat.nii.gz
36anat.nii.gz
37anat.nii.gz
38anat.nii.gz
39anat.nii.gz
3anat.nii.gz
40anat.nii.gz
41anat.nii.gz
42anat.nii.gz
43anat.nii.gz
44anat.nii.gz
45anat.nii.gz
46anat.nii.gz
47anat.nii.gz
48anat.nii.gz
49anat.nii.gz
4anat.nii.gz
50anat.nii.gz
51anat.nii.gz
52anat.nii.gz
53anat.nii.gz
54anat.nii.gz
55anat.nii.gz
56anat.nii.gz
57anat.nii.gz
58anat.nii.gz
59anat.nii.gz
5anat.nii.gz
60anat.nii.gz
61anat.nii.gz
62anat.nii.gz
63anat.nii.gz
64anat.nii.gz
65anat.nii.gz
66anat.nii.gz
67anat.nii.gz
68anat.nii.gz
69anat.nii.gz
6anat.nii.gz
70anat.nii.gz
71anat.nii.gz
72anat.nii.gz
73anat.nii.gz
74anat.nii.gz
75anat.nii.g

In [12]:
auts = np.array(auts)
cnts = np.array(cnts)

In [16]:
auts.shape

(80, 150, 200, 150)

In [17]:
cnts.shape

(80, 150, 200, 150)

In [18]:
images = np.concatenate([auts, cnts])

In [19]:
images.shape

(160, 150, 200, 150)

In [144]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l1( l=0.01),
                           input_shape=(150, 200, 150)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l1( l=0.01)),
    tf.keras.layers.MaxPooling2D((3, 3), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation=tf.nn.relu, kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

In [145]:
BATCH_SIZE = 8

In [146]:
dataset = tf.data.Dataset.from_tensor_slices( ( images, labels ) )   # tf.constant() does not change error
dataset = dataset.repeat().batch(BATCH_SIZE)

In [147]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [148]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              #metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
              #loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [149]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 200, 32)      43232     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 75, 100, 32)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 75, 100, 64)       18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 37, 49, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 116032)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 32)                3713056   
_________________________________________________________________
dense_23 (Dense)             (None, 16)               

In [150]:
model.fit(dataset, epochs=5, steps_per_epoch= 160/8)

Epoch 1/5
17/20 [========================>.....] - ETA: 12s - loss: 1253489.0000 - accuracy: 0.8235

KeyboardInterrupt: 

In [29]:
aut_test = []
cnt_test = []

In [31]:
for i in os.listdir(test_aut_path):
    im = nib.load(test_aut_path+i).get_fdata()
    #arr = to_arr(aut_path+i)
    r = skTrans.resize(im, (150,200,150), order=1, preserve_range=True)#.reshape(150,200,150,1)
    #print(r.shape)
    #break
    #if arr.shape[2] != 150:
    #    #print(arr.shape)
    #    arr = arr[:,:,46:-60]
    print(i)
    aut_test.append(r)
for i in os.listdir(test_cnt_path): 
    #arr = to_arr(cnt_path+i)
    #if arr.shape[2] != 150:
    #    #print(arr.shape)
    #    arr = arr[:,:,46:-60]
    #    #print(arr.shape)
    im = nib.load(test_cnt_path+i).get_fdata()
    r = skTrans.resize(im, (150,200,150), order=1, preserve_range=True)#.reshape(150,200,150,1)
    print(i)
    cnt_test.append(r)
    #cnts.append(to_arr(cnt_path+i).reshape(193,256, 256))

100anat.nii.gz
158anat.nii.gz
81anat.nii.gz
82anat.nii.gz
83anat.nii.gz
84anat.nii.gz
85anat.nii.gz
86anat.nii.gz
87anat.nii.gz
88anat.nii.gz
89anat.nii.gz
90anat.nii.gz
91anat.nii.gz
92anat.nii.gz
93anat.nii.gz
94anat.nii.gz
95anat.nii.gz
96anat.nii.gz
97anat.nii.gz
98anat.nii.gz
99anat.nii.gz
100anat.nii.gz
272anat.nii.gz
81anat.nii.gz
82anat.nii.gz
83anat.nii.gz
84anat.nii.gz
85anat.nii.gz
86anat.nii.gz
87anat.nii.gz
88anat.nii.gz
89anat.nii.gz
90anat.nii.gz
91anat.nii.gz
92anat.nii.gz
93anat.nii.gz
94anat.nii.gz
95anat.nii.gz
96anat.nii.gz
97anat.nii.gz
98anat.nii.gz
99anat.nii.gz


In [32]:
aut_test = np.array(aut_test)

In [98]:
aut_test.shape

(21, 150, 200, 150)

In [34]:
cnt_test = np.array(cnt_test)


In [99]:
cnt_test.shape

(21, 150, 200, 150)

In [137]:
c = 0;
for i in range(21):
    test = tf.data.Dataset.from_tensors( aut_test[i] )   # tf.constant() does not change error
    test = test.batch(1)
    r = model.predict(test)
    if np.argmax(r) == 0:
        c+=1
    print(i, np.argmax(r)==0)
print(c,'/', 21)

0 False
1 True
2 True
3 False
4 False
5 True
6 False
7 False
8 True
9 False
10 False
11 False
12 True
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
5 / 21


In [138]:
c = 0
for i in range(21):
    test = tf.data.Dataset.from_tensors( cnt_test[i] )   # tf.constant() does not change error
    test = test.batch(1)
    r = model.predict(test)
    if np.argmax(r) == 1:
        c+=1
    print(i, np.argmax(r)==1)
print(c, '/', 21)

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 False
9 False
10 True
11 True
12 False
13 False
14 True
15 True
16 True
17 True
18 False
19 True
20 False
15 / 21


In [120]:
for i in range(160):
    test = tf.data.Dataset.from_tensors( images[i] )   # tf.constant() does not change error
    test = test.batch(1)
    r = model.predict(test)
    print(i, r)

0 [[1.]]
1 [[1.]]
2 [[1.]]
3 [[1.]]
4 [[1.]]
5 [[1.]]
6 [[1.]]
7 [[1.]]
8 [[1.]]
9 [[1.]]
10 [[1.]]
11 [[1.]]
12 [[1.]]
13 [[1.]]
14 [[1.]]
15 [[1.]]
16 [[1.]]
17 [[1.]]
18 [[1.]]
19 [[1.]]
20 [[1.]]
21 [[1.]]
22 [[1.]]
23 [[0.9996703]]
24 [[1.]]
25 [[1.]]
26 [[1.]]
27 [[1.]]
28 [[1.]]
29 [[1.]]
30 [[1.]]
31 [[1.]]
32 [[1.]]
33 [[1.]]
34 [[1.]]
35 [[1.]]
36 [[1.]]
37 [[1.]]
38 [[1.]]
39 [[1.]]
40 [[1.]]
41 [[1.]]
42 [[1.]]
43 [[1.]]
44 [[1.]]
45 [[1.]]
46 [[1.]]
47 [[1.]]
48 [[1.]]
49 [[1.]]
50 [[1.]]
51 [[1.]]
52 [[1.]]
53 [[1.]]
54 [[1.]]
55 [[1.]]
56 [[1.]]
57 [[1.]]
58 [[1.]]
59 [[1.]]
60 [[1.]]
61 [[1.]]
62 [[1.]]
63 [[1.]]
64 [[1.]]
65 [[1.]]
66 [[1.]]
67 [[1.]]
68 [[2.885616e-07]]
69 [[1.]]
70 [[1.]]
71 [[1.]]
72 [[1.]]
73 [[1.]]
74 [[1.]]
75 [[1.]]
76 [[1.]]
77 [[1.]]
78 [[1.]]
79 [[1.]]
80 [[1.]]
81 [[1.]]
82 [[1.]]
83 [[1.]]
84 [[1.]]
85 [[1.]]
86 [[1.]]
87 [[1.]]
88 [[1.]]
89 [[1.]]
90 [[1.]]
91 [[1.]]
92 [[1.]]
93 [[1.]]
94 [[1.]]
95 [[1.]]
96 [[1.]]
97 [[1.]]
98 [[1.]]
99 

In [1]:
dataset = tf.data.Dataset.from_tensor_slices( ( images, labels ) )   # tf.constant() does not change error
dataset = dataset.batch(4)

NameError: name 'tf' is not defined

In [283]:
dataset = model.predict(dataset)
#print(len(ans))
#for images, labels in dataset.take(160):  # only take first element of dataset
#    numpy_images = images.numpy()
#    numpy_labels = labels.numpy()
#    print(numpy_labels)

numpy.ndarray

In [140]:
auts = []
cnts = []

for i in os.listdir(test_auth_path):
    arr = to_arr(test_auth_path+i)
    if arr.shape[2] != 150:
        #print(arr.shape)
        arr = arr[:,:,46:-60]
    auts.append( tf.image.resize(arr, [150,200]).numpy() )
    #auts.append(to_arr(aut_path+i).reshape(193,256,256))
for i in os.listdir(test_cnt_path): 
    arr = to_arr(test_cnt_path+i)
    if arr.shape[2] != 150:
        #print(arr.shape)
        arr = arr[:,:,46:-60]
        #print(arr.shape)
    cnts.append( tf.image.resize(arr, [150,200]).numpy() )
    #cnts.append(to_arr(cnt_path+i).reshape(193,256, 256))

In [141]:
auts = np.array(auts)
cnts = np.array(cnts)

In [142]:
auts.shape

(21, 150, 200, 150)

In [143]:
cnts.shape

(21, 150, 200, 150)

In [144]:
images = np.concatenate([auts, cnts])

In [145]:
images.shape

(42, 150, 200, 150)

In [146]:
test = tf.data.Dataset.from_tensor_slices( ( images, test_labels ) )   # tf.constant() does not change error
test = test.batch(1)

In [147]:
test

<BatchDataset shapes: ((None, 150, 200, 150), (None,)), types: (tf.float32, tf.int32)>

In [148]:
ans = model.predict(test)
for i in ans:
    print(np.argmax(ans))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
for i in range(400*2):
  test = tf.data.Dataset.from_tensors( images[i] )   # tf.constant() does not change error
  test = test.batch(1)
  r = model.predict(test)
  print(i, np.argmax(r))

In [ ]:
a = np.array([1,0])
np.argmax(a)

In [ ]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
#images, labels = train
#images = images/255

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.cache()

In [ ]:
labels.shape

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
for img, label in dataset.take(460):
  test_images = img.numpy()
  test_labels = label.numpy()
  #print(test_images.shape)
  #print(test_labels.shape)

In [ ]:
test_labels

In [ ]:
test_images.shape

In [ ]:
plt.imshow(test_images.reshape(28,28))
print(test_labels)
print('prediction:',model.predict(images[90].reshape(1,28,28,1)))

In [ ]:
for i in range(126):
  print(labels[i], " >>", model.predict( images[i].reshape(1, 28,28,1) ))

In [ ]:
img, label = dataset.take(1)

ModuleNotFoundError: No module named 'nilearn'